<a href="https://colab.research.google.com/github/chriskwon9/deeplearning/blob/main/kyungheenlp_0923_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim

from collections import defaultdict
import time
import random
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence

In [21]:
# Functions to read in the corpus
w2i = defaultdict(lambda: len(w2i))
t2i = defaultdict(lambda: len(t2i))
UNK = w2i["<unk>"]

In [22]:
def read_dataset(filename):
  with open(filename, "r") as f:
    for line in f:
      tag, words = line.lower().strip().split(" ||| ")
      yield ([w2i[x] for x in words.split(" ")], t2i[tag])

In [23]:
from google.colab import files
uploaded = files.upload()

Saving train.txt to train (1).txt


In [24]:
from google.colab import files
uploaded = files.upload()

Saving test.txt to test (1).txt


In [25]:
# Read in the data
train_data = list(read_dataset("train.txt"))
w2i = defaultdict(lambda: UNK, w2i)  # Now UNK is used for unknown words
dev_data = list(read_dataset("test.txt"))
nwords = len(w2i)
ntags = len(t2i)

In [26]:
# Custom dataset class to work with DataLoader
class TextDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [27]:
# Collate function for padding sentences in each batch
def collate_fn(batch):
    sentences, tags = zip(*batch)
    sentences = [torch.tensor(sent) for sent in sentences]
    tags = torch.tensor(tags)
    padded_sentences = pad_sequence(sentences, batch_first=True, padding_value=UNK)
    return padded_sentences, tags

In [28]:
# Define DataLoader
BATCH_SIZE = 32
train_loader = DataLoader(TextDataset(train_data), batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
dev_loader = DataLoader(TextDataset(dev_data), batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

In [29]:
# Define the model
EMB_SIZE = 64
HID_SIZE = 64

In [30]:
class RNNModel(nn.Module):
    def __init__(self, nwords, ntags, emb_size, hidden_size):
        super(RNNModel, self).__init__()
        self.embedding = nn.Embedding(nwords, emb_size)
        self.rnn = nn.RNN(emb_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, ntags)

        # Initialize weights
        self.init_weights()

    def init_weights(self):
        # Apply Xavier initialization to all linear and recurrent layers
        for name, param in self.named_parameters():
            if 'weight' in name:
                nn.init.xavier_uniform_(param.data)
            elif 'bias' in name:
                param.data.fill_(0)  # Initialize biases to zero

    def forward(self, sentences):
        embeds = self.embedding(sentences)  # [batch_size x len(sentences[1]) x emb_size]
        rnn_out, _ = self.rnn(embeds)       # [batch_size x len(sentences[1]) x hidden_size]
        logits = self.fc(rnn_out[:, -1, :]) # Use the last hidden state for classification
        return logits

model = RNNModel(nwords, ntags, EMB_SIZE, HID_SIZE)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [33]:
# Training loop
max_test_accuracy = 0.0
for ITER in range(100):
    # Perform training
    train_loss = 0.0
    start = time.time()
    model.train()

    for sentences, tags in tqdm(train_loader):
        logits = model(sentences)
        loss = criterion(logits, tags)
        train_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"iter {ITER}: train loss/sent={train_loss / len(train_loader):.4f}, time={time.time() - start:.2f}s")

    # Perform evaluation
    model.eval()
    test_correct = 0.0
    with torch.no_grad():
        for sentences, tags in tqdm(dev_loader):
            logits = model(sentences)
            predict = torch.argmax(logits, dim=1)
            test_correct += (predict == tags).sum().item()

    test_accuracy = test_correct / len(dev_data)
    if max_test_accuracy < test_accuracy:
        max_test_accuracy = test_accuracy
    print(f"iter {ITER}: test acc={test_accuracy:.4f}")

print("max test acc=%.4f" % (max_test_accuracy))

100%|██████████| 267/267 [00:06<00:00, 43.16it/s]


iter 0: train loss/sent=1.5762, time=6.19s


100%|██████████| 70/70 [00:00<00:00, 321.54it/s]


iter 0: test acc=0.2443


100%|██████████| 267/267 [00:04<00:00, 57.70it/s]


...

iter 98: test acc=0.3163


100%|██████████| 267/267 [00:04<00:00, 56.32it/s]


iter 99: train loss/sent=1.4960, time=4.75s


100%|██████████| 70/70 [00:00<00:00, 479.84it/s]

iter 99: test acc=0.2579
max test acc=0.3195
